# Rasgos

Las posibilidades que ofrece el despacho múltiple en combinación con la riqueza ilimitada del sistema de tipos son enormes.

Uno de los patrones más comunes en el desarrollo de código en Julia es el de los **Tim Holy traits** o **rasgos**, nombrado así en honor a su [descubridor](https://github.com/JuliaLang/julia/issues/2345#issuecomment-54537633) (¿o creador...?).

Los rasgos representan comportamientos que un tipo puede tener. En un ejemplo como el del Notebook anterior, tanto un `Aguila` como un `Murcielago` pueden volar y caminar, por lo que ambos tendrían  los rasgos *PuedeVolar* y *PuedeCaminar*. Un pingüino, a pesar de ser un ave, puede caminar y nadar, pero no puede volar. En cambio, un `Pato` puede volar, nadar y caminar. Típicamente, los rasgos son binarios: un tipo puede tenerlos o no tenerlos.

Los rasgos ofrecen una forma de extender y complejizar el sistema de tipos y sus comportamientos sin tener que modificar la base preexistente. Para ver las ventajas en un ejemplo concreto, retomemos el ejemplo del Notebook anterior: 

In [2]:
abstract type Animal end

abstract type Mamifero <: Animal end
abstract type Ave <: Animal end
abstract type Reptil <: Animal end
abstract type Anfibio <: Animal end
abstract type Insecto <: Animal end

In [3]:
mutable struct Aguila <: Ave
    x::Float64
    y::Float64
    z::Float64
end

mutable struct Murcielago <: Mamifero
    x::Float64
    y::Float64
    z::Float64
end

mutable struct Pinguino <: Ave
    x::Float64
    y::Float64
end

Supongamos que queremos definir una función

```julia
volar!(a::Animal, pos::Vector{Float64})
```

que transporte el animal `a` volando a una nueva posición `pos`, si es que el animal puede volar, y que falle en otro caso. No podemos usar el despacho múltiple directamente sobre los tipos que ya hemos definido porque, por ejemplo, como vimos, la capacidad de volar no tiene relación con la jerarquía de tipos establecida: algunas aves pueden volar, pero no todas, y algunos mamíferos pueden volar, pero no todos. 

Una primera solución rudimentaria podría ser introducir una sentencia `if` dentro de la función `volar!` que verifique si el animal puede volar o no:

In [4]:
puede_volar(a::Animal) = false # Por defecto, no puede volar
puede_volar(a::Aguila) = true

function volar_if!(a::Animal, pos::Vector{Float64})
    if puede_volar(animal)
        animal.x = pos[1]
        animal.y = pos[2]
        println("Volando a ", pos)
    else
        error("No puede volar")
    end
end

volar_if! (generic function with 1 method)

Sin embargo, esta solución no es buena porque implica el `if` se resuelve en tiempo de ejecución, cuando queremos que la función compile a una versión lo más específica, y por lo tanto rápida, posible.

Lo mejor es usar los tipos para hacer despacho. Otra posible solución en esa línea es usar un tipo `Union` sobre los animales que pueden volar para despachar la funcion `volar!` a un método específico.

In [5]:
function volar_Union!(a::Animal, pos::Vector{Float64}) # Por defecto, no puede volar
    error("No puede volar")
end

function volar_Union!(a::Union{Aguila, Murcielago}, pos::Vector{Float64})
    animal.x = pos[1]
    animal.y = pos[2]
    println("Volando a ", pos)
end

volar_Union! (generic function with 2 methods)

Esta solución es mejor que la anterior, pero no es óptima porque para cada nuevo tipo de animal que queramos agregar, debemos acordarnos de incorporarlo a la `Union` y modificar la función `volar!`.

La solución óptima es introducir rasgos, que serán tipos binarios: o se tiene el rasgo o no se lo tiene. Generalmente se los define de la siguiente manera:

In [7]:
abstract type CapacidadDeVuelo end

struct PuedeVolar <: CapacidadDeVuelo end
struct NoPuedeVolar <: CapacidadDeVuelo end

y asignamos los rasgos a los distintos tipos

In [9]:
capacidad_de_vuelo(a::Animal) = NoPuedeVolar() # Por defecto, no pueden volar
capacidad_de_vuelo(a::Aguila) = PuedeVolar()

capacidad_de_vuelo (generic function with 2 methods)

Ahora, podemos definir la función `volar!` introduciendo una redirección a otros métodos que dependan del rasgo:

In [10]:
volar!(a::Animal, pos::Vector{Float64}) = volar!(capacidad_de_vuelo(a), a, pos)

volar! (generic function with 1 method)

Y definimos los métodos como:

In [11]:
function volar!(::PuedeVolar, animal::Animal, pos::Vector{Float64})
    animal.x = pos[1]
    animal.y = pos[2]
    println("Volando a ", pos)
end

function volar!(::NoPuedeVolar, animal::Animal, pos::Vector{Float64})
    error("No puede volar")
end

volar! (generic function with 3 methods)

Ahora, cuando queremos agregar un nuevo animal volador al sistema de tipos, simplemente debemos asignarle el rasgo `PuedeVolar` o `NoPuedeVolar` a través de la función `capacidad_de_volar`. No necesitamos modificar un tipo Union ni la función `volar!`, la base de código preexistente permanece igual.

In [16]:
capacidad_de_vuelo(a::Murcielago) = PuedeVolar()

capacidad_de_vuelo (generic function with 1 method)

Esto es especialmente ventajoso cuando desarrollamos un paquete para ser utilizado por otros usuarios. Si queremos que los usuarios puedan agregar sus propios animales voladores, por ejemplo, el tipo `Union` no estaría disponible para extensión, mientras que con el uso de rasgos solo necesitarán implementar el método `capacidad_de_volar` para ese tipo asignándole el rasgo `PuedeVolar()`.